# Data Encryption Standard (DES) Python Implementation
U.S. Department of Commerce. (1999). FIPS PUB 46-3: Data Encryption Standard (DES). Available at: https://csrc.nist.gov/publications/detail/fips/46/3/archive/1999-10-25 [Accessed 7 February 2020].

## Overview
A. Base algorithm
  1. Key schedule calculation (KSC): Generates 48 bit keys for use in iterations.
  2. Calculation of f(R, K): Expansion, xor s-box diffusion.
  3. Enciphering computation: 16 iterations
  4. Deciphering computation: 16 iterations

B. Modes of Operation
  1. Electronic Code Book (ECB) Mode
  2. Cipher Block Chain (CBC) Mode

### Import and Script Operation Variables

In [1]:
import sys

In [2]:
debug = 1 # Debug level 0 - 3, where 0 is no debug messages
#info_level = 2

### General Support Functions

In [3]:
# Custom print formatting
def cPrint(str, format):
    out = str
    if format == "underline":
        out = "\033[4m"+str+"\033[0m"
    elif format == "bold":
        out = "\033[1m"+str+"\033[0m"
    print(out)

    
# Binary, Hex, Ascci Conversion
def convert_hex_to_binary(hexdigits):
    binarydigits = ""
    for hexdigit in hexdigits:
        binarydigits += bin(int(hexdigit,16))[2:].zfill(4)
    return binarydigits

def convert_binary_to_hex(binarydigits):
    hexdigits = '%0*X' % ((len(binarydigits) + 3) // 4, int(binarydigits, 2))
    return hexdigits

def convert_ascii_to_hex(ascii_input):
    return ''.join([hex(ord(c))[2:].zfill(2) for c in ascii_input])

def convert_hex_to_ascii(hex_input):
    return ''.join([chr(int(''.join(c), 16)) for c in zip(hex_input[0::2],hex_input[1::2])])


# xor
def xor(input1,input2):
    # xor
    # Example: xor(input1,input2)
    
    result = ""
    for index in range(len(input1)):
        if input1[index] == input2[index]: 
            result += '0'
        else:
            result += '1'
    return result

## Key schedule calculation (KSC)
Figure 3.Key schedule calculation. U.S. Department of Commerce. (1999). FIPS PUB 46-3: Data Encryption Standard (DES), pp.20.


<img src="./assets/des_figure_3_key_schedule_calculation.png" align="left" width="40%" height=auto>

### Static Variables for KSC

In [4]:
permuted_choice_1_table = [57,49,41,33,25,17,9,
                           1,58,50,42,34,26,18,
                           10,2,59,51,43,35,27,
                           19,11,3,60,52,44,36,
                           63,55,47,39,31,23,15,
                           7,62,54,46,38,30,22,
                           14,6,61,53,45,37,29,
                           21,13,5,28,20,12,4]

permuted_choice_2_table = [14,17,11,24,1,5,
                           3,28,15,6,21,10,
                           23,19,12,4,26,8,
                           16,7,27,20,13,2,
                           41,52,31,37,47,55,
                           30,40,51,45,33,48,
                           44,49,39,56,34,53,
                           46,42,50,36,29,32]

iteration_shifts = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

### KSC Functions

In [5]:
# KSC Function-1 Permutations
def apply_permutation(permuted_choice_table,ini_input):
    # Function takes an initial key (ini_key) and permutes using static permuted_choice_table
    # returning key of len(permuted_choice_table) bits  as output (gen_key).
    # Example usage: out_key56 = apply_permutation(apply_permutation_choice_1)
    
    permutated_out = ""
    for index in permuted_choice_table:
        permutated_out += ini_input[index-1] # -1 as list index starts at 0
    return permutated_out


# KSC Function-2. Split inout into two equal length parts
def split_in_half(initial_input,initial_length):
    # Function takes a n bit input and splits into two half based on initial_length/2
    # Initial_input and initial_length must be an even number
    # Example usage: Ci , Di = split_in_half(out_key56, 56)
    
    split = int(initial_length/2)
    left, right = initial_input[:split],initial_input[split:]
    return left, right


# KSC Function-3. Circular left shift
def circular_left_shift(input_bits,n_shift):
    # Function applies n (n_shift) circular shift to input (input_bits)
    # Example usage:circularShiftedBits = circular_left_shift(bits,numberofbits)
    
    shifted_bits = input_bits[n_shift:] + input_bits[:n_shift]
    return shifted_bits

### Key schedule calculation

In [6]:
def gen_schedule_keys(k):
    # Initialise list to load generated keys for use in the 16 rounds 
    schedule_keys = list()

    # Check initial k length
    if len(k) != 64:
        print("Initial key (k) must be 64 bits, currently it is "+str(len(k))+" bits.")
        sys.exit(0)

    # Permuted Choice 1: Table permutation & Key Split
    out_key56 = apply_permutation(permuted_choice_1_table,k) 
    C0, D0 = split_in_half(out_key56, 56)

    # Repeat 16 times
    for i in range(16):

        # Circular Left Shifts
        Ci = circular_left_shift(C0,iteration_shifts[i])
        Di = circular_left_shift(D0,iteration_shifts[i])

        # Permuted Choice 2: Table permutation
        Kn = apply_permutation(permuted_choice_2_table,Ci+Di)

        # Add Kn to schedule_keys list
        schedule_keys.append(Kn)

        # Check this - Set the key to be shiften in the next round to the shifted key in current round
        C0 = Ci
        D0 = Di
    return schedule_keys

# The schedule_keys will have all 16 keys as Python list
# print(schedule_keys)


## Calculation of f(R, K)
Figure 2.Calculation of f(R, K). U.S. Department of Commerce. (1999). FIPS PUB 46-3: Data Encryption Standard (DES), pp.13.

<img src="./assets/des_figure_2_calculation_of_cipher_function_f.png" align="left" width="40%" height=auto>

### Static Variables for calculation of f(R, K)

In [7]:
ebit_selection_table = [32,1,2,3,4,5,
                        4,5,6,7,8,9,
                        8,9,10,11,12,13,
                        12,13,14,15,16,17,
                        16,17,18,19,20,21,
                        20,21,22,23,24,25,
                        24,25,26,27,28,29,
                        28,29,30,31,32,1]

sbox_tables = [
    # S1
    [[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
     [0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
     [4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
     [15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]],

    # S2
    [[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
     [3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
     [0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
     [13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]],

    # S3
    [[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
     [13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
     [13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
     [1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]],

    # S4
    [[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
     [13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
     [10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
     [3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]],

    # S5
    [[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
     [14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
     [4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
     [11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]],

    # S6
    [[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
     [10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
     [9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
     [4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]],

    # S7
    [[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
     [13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
     [1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
     [6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]],

    # S8
    [[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
     [1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
     [7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
     [2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]]
    ]

p_table = [16,7,20,21,
           29,12,28,17,
           1,15,23,26,
           5,18,31,10,
           2,8,24,14,
           32,27,3,9,
           19,13,30,6,
           22,11,4,25]

### Functions for Calculation of f(R, K) 

In [8]:
# f(R, K) Function-1 S-Box Lookup
def sbox_lookup(sbox_lookup_table_number,first_last_bits,middle4_bits):
    # Function looks up
    
    # Convert first & last and middle 4 bit variables to decimal
    first_last_bits_as_decimal = int(first_last_bits,2)
    middle4_bits_as_decimal = int(middle4_bits,2)
    
    #Lookup in sbox tables
    sbox_value = sbox_tables[sbox_lookup_table_number][first_last_bits_as_decimal][middle4_bits_as_decimal]
    
    # Return binary
    return bin(sbox_value)[2:].zfill(4)


### Calculation of f(R, K) 

In [9]:
def f_function(R,key48):
    # Initialise result
    result = ""

    # E expansion
    #left_expanded48 = apply_e_expansion(ebit_selection_table,R)
    #e_expansion is just a permutation using ebit_selection_table
    left_expanded48 = apply_permutation(ebit_selection_table,R)

    # xor left expansion with key 
    xor_result = xor(left_expanded48,key48)

    # Split xor result into 6 bit segments
    split_into6bit_list = [xor_result[i:i+6] for i in range(0, len(xor_result), 6)]

    for sbox_lookup_table_number, bits6_section in enumerate(split_into6bit_list):

        first_last_bits = bits6_section[0] + bits6_section[-1]

        middle4_bits = bits6_section[1:5]

        sbox_result = sbox_lookup(sbox_lookup_table_number,first_last_bits,middle4_bits)

        result += sbox_result

    final32bits = apply_permutation(p_table,result)
    return final32bits

## Enciphering computation (Feistel function)
Figure 1. Enciphering computation. U.S. Department of Commerce. (1999). FIPS PUB 46-3: Data Encryption Standard (DES), pp.9.

<img src="./assets/des_figure_1_enciphering_computation.png" align="left" width="40%" height=auto>

### Static Variables for enciphering

In [10]:
initial_permutation_table = [58, 50, 42, 34, 26, 18, 10, 2,
                             60, 52, 44, 36, 28, 20, 12, 4,
                             62, 54, 46, 38, 30, 22, 14, 6,
                             64, 56, 48, 40, 32, 24, 16, 8,
                             57, 49, 41, 33, 25, 17, 9, 1,
                             59, 51, 43, 35, 27, 19, 11, 3,
                             61, 53, 45, 37, 29, 21, 13, 5,
                             63, 55, 47, 39, 31, 23, 15, 7]

inverse_permutation_table = [40, 8, 48, 16, 56, 24, 64, 32,
                             39, 7, 47, 15, 55, 23, 63, 31,
                             38, 6, 46, 14,  54, 22, 62, 30,
                             37, 5, 45, 13, 53, 21, 61, 29,
                             36, 4, 44, 12, 52, 20, 60, 28,
                             35, 3, 43, 11, 51, 19, 59, 27, 
                             34, 2, 42, 10, 50, 18, 58, 26,
                             33, 1, 41, 9, 49, 17, 57, 25]


### Enciphering

In [11]:
def encrypt_des(m,k):
    # Function takes message and key in hex
    
    # Convert hex digits to binary
    m_in_bits = convert_hex_to_binary(m)
    key_in_bits = convert_hex_to_binary(k)
    
    if debug == 3:
        print("\n[DEBUG]\t","Starting encryption [encrypt_des]")
        print("[DEBUG]\t","m in bits:",m_in_bits)
        print("[DEBUG]\t","k in bits:",key_in_bits)
    
    # Generate schedule keys for use in iterations
    schedule_keys = gen_schedule_keys(key_in_bits)

    # Initial permutation
    permutated_m = apply_permutation(initial_permutation_table,m_in_bits)
    
    # split permutated message into two
    L0,R0 = split_in_half(permutated_m,64)
    
    # Initialise left and right halves of permutated message for use in iterations
    L = L0
    R = R0
    
    # Loop for 16 iterations
    if debug == 3:
            print("\n[DEBUG]\t","Starting Iterations")
    for i in range(16):
        if debug == 3:
            print("[DEBUG]\t","Iteration:",i," schedule_key:",schedule_keys[i])

        Rn1 = xor(L,f_function(R, schedule_keys[i]))
        Ln1 = R
        
        R = Rn1
        L = Ln1
    
    # Final inverse initial permutation
    c = apply_permutation(inverse_permutation_table, R+L)
    
    return convert_binary_to_hex(c)


if debug >= 2:
    cPrint("\n[DEBUG]\tTesting encrypt_des with static hex variables\n", "bold")
    m = 'A1B2C3D4E5F67890' # Hex message 16 bits / 64 bit binary
    k = '01AF5E32134C8FA3'

    # Show results
    cPrint("[DEBUG]\tInput", "underline")
    print("[DEBUG]\tInitial Message (m):\t\t",m)
    print("[DEBUG]\tKey:\t\t\t\t",k)

    c = encrypt_des(m,k)
    
    print("\r")
    cPrint("[DEBUG]\tOutput", "underline")
    print("[DEBUG]\tCipher in hexadecimal:\t\t", c)


# Deciphering

In [12]:
def decrypt_des(c,k):
    # Function takes cipher text (c) and key (k) as hexadecimal and decrypts c with k.
    
    c_in_bits = convert_hex_to_binary(c)
    key_in_bits = convert_hex_to_binary(k)
    
    if debug == 3:
        print("\n[DEBUG]\t","Starting decryption [decrypt_des]")
        print("[DEBUG]\t","c in bits:",c_in_bits)
        print("[DEBUG]\t","k in bits:",key_in_bits)
    
    # Generate schedule keys for use in iterations
    schedule_keys = gen_schedule_keys(key_in_bits)
    
    # Final inverse initial permutation
    permutated_c = apply_permutation(initial_permutation_table, c_in_bits)

    
    # split permutated message into two
    R0, L0 = split_in_half(permutated_c,64)
    
    # Initialise left and right halves of permutated message for use in iterations
    L = L0
    R = R0
    
    # Loop for 16 iterations
    if debug == 3:
        print("\n[DEBUG]\t","Starting Iterations")
        
    for i in reversed(range(16)):
        if debug == 3:
            print("[DEBUG]\t","Iteration:",i," schedule_key:",schedule_keys[i])
        
        Rn1 = L
        Ln1 = xor(R,f_function(L, schedule_keys[i]))
        
        R = Rn1
        L = Ln1
        
    # Initial permutation
    decrypted_m = apply_permutation(inverse_permutation_table,L+R)
    
    return convert_binary_to_hex(decrypted_m)

if debug >= 2:
    cPrint("\n[DEBUG]\tTesting decrypt_des with static hex variables\n", "bold")
    m = "A1B2C3D4E5F67890"
    k = '01AF5E32134C8FA3'
    c = 'E2D58918E777331C'

    # Show results
    cPrint("[DEBUG]\tInput", "underline")
    print("[DEBUG]\tInitial Message (m):\t\t",m)
    print("[DEBUG]\tCipher:\t\t\t\t",c)
    print("[DEBUG]\tKey:\t\t\t\t",k)

    decrypted_m = decrypt_des(c,k)
    
    print("\r")
    cPrint("[DEBUG]\tOutput", "underline")
    print("[DEBUG]\tDecrypted m in hexadecimal:\t", decrypted_m)

# Modes of Operation

### Padding Functions

In [13]:
# Padding scheme 1: Append a single byte with value of 128 (decimal) (80 hex),
# then add as many as zero bytes as required to complete block

def apply_padding_s1(hex_m):
    hex_m_padded = hex_m
    rem = len(hex_m) % 16
    
    if rem > 0:
        # add padding
        padding_len = 16-rem
        if padding_len == 2:
            hex_m_padded = hex_m+"80" # 80 = hex for 128
        elif padding_len > 2:
            no_of_zeros = (padding_len-2)
            hex_m_padded = hex_m+"80"
            for i in range(no_of_zeros):
                hex_m_padded = hex_m_padded+"0"         
                
    return hex_m_padded

def remove_padding_s1(hex_m_padded):
    # NEEDS WORK - how to check- careful not to split a valid 128 dec (80 in hex) characters
    hex_m = hex_m_padded.rsplit('80', 1)
    return hex_m[0]


# Testing padding functions
if debug >= 2:
    m1 = "ABCDEFGH"
    m2 = "ABCDEFGHI"
    m3 = "ABCDEFGHIJK"
    
    cPrint("Test Padding Functions", "bold")
    print("\r")
    
    cPrint("m1", "underline")
    print("ascii:\t\t\t", m1)
    print("hex:\t\t\t", convert_ascii_to_hex(m1))
    hex_m1_padded = apply_padding_s1(convert_ascii_to_hex(m1))
    print("hex padded:\t\t", hex_m1_padded)
    print("\r")
    
    cPrint("m2", "underline")
    print("ascii:\t\t\t", m2)
    print("hex:\t\t\t", convert_ascii_to_hex(m2))
    hex_m2_padded = apply_padding_s1(convert_ascii_to_hex(m2))
    print("hex padded:\t\t", hex_m2_padded)
    print("\r")
    
    cPrint("m3", "underline")
    print("ascii:\t\t\t", m3)
    print("hex:\t\t\t", convert_ascii_to_hex(m3))
    hex_m3_padded = apply_padding_s1(convert_ascii_to_hex(m3))
    print("hex padded:\t\t", hex_m3_padded)
    print("hex padding removed:\t", remove_padding_s1(hex_m3_padded))
    print("ascii padding removed:\t", convert_hex_to_ascii(remove_padding_s1(hex_m3_padded)))
    print("\r")

### Block Functions

In [14]:
def split_into_blocks(input_hex, block_length):
    hex_blocks = [input_hex[i:i+block_length] for i in range(0, len(input_hex), block_length)]
    return hex_blocks

### Electronic Code Book (ECB) Mode

In [15]:
# Input m (ascii) & k (hex)
k = '01AF5E32134C8FA3' # hex 8x2 (16) char / 64 bits
# m = "Hello World!" # n lenght ascii

# Example of why not to use ECB:
m = 'abababababababab' # yields c = AACAFB4BFD8168DD AACAFB4BFD8168DD - Repeating block pattern


# Begin ECB Mode encryption
if debug >= 2:
    cPrint("Encrypting", "bold")
    print("\r")
    
# Convert ascci m to hex
hex_m = convert_ascii_to_hex(m)

# Add padding scheme 1 to hex m
hex_m_padded= apply_padding_s1(hex_m)

# Split padded hex into blocks of 8 bytes (8 x 2 long (16) hex characters )
hex_m_blocks = split_into_blocks(hex_m_padded, 16)

if debug >= 2:
    cPrint("m", "underline")
    print(m)
    print("\r")
    
    cPrint("m Hex", "underline")
    print(hex_m)
    print("\r")
    
    cPrint("m Hex Blocks (Post Padding)", "underline")
    print(hex_m_blocks)
    print("\r")

# Initialise cipher text c
c=''

# Loop hex blocks and encrypt
for block in hex_m_blocks:
    if debug >= 3:
        print("\n[DEBUG]\t","Starting new block:",block)
    
    c += encrypt_des(block,k)
    
if debug >= 2:
    cPrint("Cipher text (Hex)", "underline")
    print(c)
    print("\r")

    
####################################################################################
    

# Begin ECB Mode decryption
if debug >= 2:
    print("\r")
    cPrint("Decrypting", "bold")
    print("\r")

# Split c into blocks for decryption
hex_c_blocks = split_into_blocks(c, 16)

if debug >= 2:
    cPrint("c Hex Blocks", "underline")
    print(hex_c_blocks)
    print("\r")

# Initialise decrypted message
decrypted_m = ''

# Loop hex blocks and encrypt
for block in hex_c_blocks:
    if debug >= 3:
        print("\n[DEBUG]\t","Starting new block:",block)
        
    decrypted_m += decrypt_des(block,k)
    
    
if debug >= 2:
    cPrint("Decrypted m (Padded) (Hex)", "underline")
    print(decrypted_m)
    print("\r")
    
    cPrint("Decrypted m (Hex)", "underline")
    print(remove_padding_s1(decrypted_m))
    print("\r")
    
    cPrint("Decrypted m (ascii)", "underline")
    print(convert_hex_to_ascii(remove_padding_s1(decrypted_m)))
    print("\r")


### Cipher Block Chain (CBC) Mode

In [16]:
# Input m (ascii) & k (hex)
k = '01AF5E32134C8FA3' # hex 8x2 (16) char / 64 bits
iv = '3729F3A8E332ECC7' # hex 8x2 (16) char / 64 bits
m = "Hello World! This is a test of DES encryption" # n lenght ascii

# ECB Example with CBC:
# m = 'abababababababab' # yields c = C549D346632BF31F 80DC7FFF1F7FFFB1 - block chain mode


# Begin ECB Mode encryption
if debug >= 1:
    cPrint("Encrypting", "bold")
    print("\r")
    
# Convert ascci m to hex
hex_m = convert_ascii_to_hex(m)

# Add padding scheme 1 to hex m
hex_m_padded= apply_padding_s1(hex_m)

# Split padded hex into blocks of 8 bytes (8 x 2 long (16) hex characters )
hex_m_blocks = split_into_blocks(hex_m_padded, 16)

if debug >= 1:
    cPrint("m", "underline")
    print(m)
    print("\r")
    
    cPrint("m Hex", "underline")
    print(hex_m)
    print("\r")
    
    cPrint("m Hex Blocks (Post Padding)", "underline")
    print(hex_m_blocks)
    print("\r")

# Initialise cipher text c
c=''

transformer = iv

# Loop hex blocks and encrypt
for block in hex_m_blocks:
    if debug >= 3:
        print("\n[DEBUG]\t","Starting new block:",block)
    
    # convert hex to binary for xor
    transformer_in_binary = convert_hex_to_binary(transformer)
    block_in_binary = convert_hex_to_binary(block)
    
    # xor
    transformed_block_binary = xor(transformer_in_binary,block_in_binary)
    
    # convert back to hex so encrypt_des will accept input
    transformed_block_hex = convert_binary_to_hex(transformed_block_binary)
    
    #Encrypt block 
    c_block = encrypt_des(transformed_block_hex,k)
    
    # Append block 
    c += c_block
    
    # Set the cipher of current block to be used as transformer in next block
    transformer = c_block
    
    
if debug >= 1:
    cPrint("Cipher text (Hex)", "underline")
    print(c)
    print("\r")

    
####################################################################################
    

# Begin ECB Mode decryption
if debug >= 1:
    print("\r")
    cPrint("Decrypting", "bold")
    print("\r")

# Split c into blocks for decryption
hex_c_blocks = split_into_blocks(c, 16)

if debug >= 1:
    cPrint("c Hex Blocks", "underline")
    print(hex_c_blocks)
    print("\r")

# Initialise decrypted message
decrypted_m = ''

# Loop hex blocks and encrypt
for block in reversed(hex_c_blocks):
    if debug >= 3:
        print("\n[DEBUG]\t","Starting new block:",block)
        
    if hex_c_blocks.index(block) == 0:
        # use static iv
        transformer = iv
        
    else:
        # use cn-1
        transformer = hex_c_blocks[hex_c_blocks.index(block)-1]
    
    # Decrypt block 
    d_block = decrypt_des(block,k)
    
    # convert hex to binary for xor
    transformer_in_binary = convert_hex_to_binary(transformer)
    d_block_in_binary = convert_hex_to_binary(d_block)
    
    # xor
    transformed_d_block_binary = xor(transformer_in_binary,d_block_in_binary)
    
    # convert back to hex so encrypt_des will accept input
    transformed_d_block_hex = convert_binary_to_hex(transformed_d_block_binary)
    
    # Append block 
    decrypted_m = transformed_d_block_hex + decrypted_m
    
    
if debug >= 1:
    cPrint("Decrypted m (Padded) (Hex)", "underline")
    print(decrypted_m)
    print("\r")
    
    cPrint("Decrypted m (Hex)", "underline")
    print(remove_padding_s1(decrypted_m))
    print("\r")
    
    cPrint("Decrypted m (ascii)", "underline")
    print(convert_hex_to_ascii(remove_padding_s1(decrypted_m)))
    print("\r")

Encrypting

m
Hello World! This is a test of DES encryption

m Hex
48656c6c6f20576f726c6421205468697320697320612074657374206f662044455320656e6372797074696f6e

m Hex Blocks (Post Padding)
['48656c6c6f20576f', '726c642120546869', '7320697320612074', '657374206f662044', '455320656e637279', '7074696f6e800000']

Cipher text (Hex)
8963FCACD422C99BE82A32FE7B81305AD8B827EA8C98833B295E5A37E88204DF71B533C38E39B2A95643C1C44B8BE32B


Decrypting

c Hex Blocks
['8963FCACD422C99B', 'E82A32FE7B81305A', 'D8B827EA8C98833B', '295E5A37E88204DF', '71B533C38E39B2A9', '5643C1C44B8BE32B']

Decrypted m (Padded) (Hex)
48656C6C6F20576F726C6421205468697320697320612074657374206F662044455320656E6372797074696F6E800000

Decrypted m (Hex)
48656C6C6F20576F726C6421205468697320697320612074657374206F662044455320656E6372797074696F6E

Decrypted m (ascii)
Hello World! This is a test of DES encryption

